In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("bank_churn.csv")
df.shape

(10000, 14)

In [2]:
df.sample(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
4961,4962,15632521,Cattaneo,689,Germany,Male,45,0,130170.82,2,1,0,150856.38,0
9828,9829,15574523,Cheng,576,France,Male,39,1,0.00,2,1,1,68814.23,0
9652,9653,15726720,Blinova,480,France,Female,40,7,0.00,1,1,0,170332.67,1


In [3]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [4]:
df.drop(['RowNumber','CustomerId','Surname'],axis='columns',inplace=True)
df.replace('Male',1,inplace=True)
df.replace('Female',0,inplace=True)
df_dummy = pd.get_dummies(df.Geography)
df_dummy.drop("Spain",axis='columns',inplace=True)
df_new = pd.concat([df,df_dummy],axis='columns')
df_new.drop("Geography",axis='columns',inplace=True)
df_new.sample(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Germany
8499,498,1,35,2,121968.11,2,0,1,188343.05,0,0,1
9142,494,0,35,9,112727.06,2,1,0,183752.91,0,1,0
3783,647,0,26,8,109958.15,1,1,1,136592.24,1,0,0
6551,525,0,30,5,0.00,2,0,1,149195.44,0,0,0
323,763,0,31,4,0.00,2,0,0,50404.72,0,1,0


In [5]:
#Scaling
columns_to_scale = ['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary']
from sklearn.preprocessing import  MinMaxScaler
scaler = MinMaxScaler()
df_new[columns_to_scale] = scaler.fit_transform(df_new[columns_to_scale])

In [6]:
x = df_new.drop("Exited",axis='columns')
y = df.Exited
x.shape,y.shape

((10000, 11), (10000,))

In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=15)

In [8]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report

In [9]:
def ANN(x_train_ANN,x_test_ANN,y_train_ANN,y_test_ANN,loss_ANN,weights_ANN):
    model = keras.Sequential([
        keras.layers.Input(shape=(11,)),
        keras.layers.Dense(100,activation='relu'),
        keras.layers.Dense(50,activation='relu'),
        keras.layers.Dense(1,activation='sigmoid')
    ])
    model.compile(optimizer='adam',loss=loss_ANN,metrics=['accuracy'])
    if weights_ANN==-1:
        model.fit(x_train_ANN,y_train_ANN,epochs=100)
    else:
        model.fit(x_train_ANN,y_train_ANN,epochs=100,class_weight=weights_ANN)
    print(model.evaluate(x_test_ANN,y_test_ANN))
    y_preds_ANN = model.predict(x_test_ANN)
    y_preds_ANN = np.round(y_preds_ANN)
    print("Classification report: ",classification_report(y_test_ANN,y_preds_ANN))
    return y_preds_ANN

In [26]:
y_pred = ANN(x_train,x_test,y_train,y_test,'binary_crossentropy',-1)

Epoch 1/100
250/250 [==============================] - 2s 3ms/step - loss: 0.4764 - accuracy: 0.7959
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4278 - accuracy: 0.8170
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3929 - accuracy: 0.8317
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3696 - accuracy: 0.8440
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3580 - accuracy: 0.8506
Epoch 6/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3548 - accuracy: 0.8535
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3508 - accuracy: 0.8569
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3487 - accuracy: 0.8583
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3442 - accuracy: 0.8581
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3457 - accura

# METHOD 1: UNDER SAMPLING MAJORITY CLASS

In [27]:
df_under = x_train.copy()
df_under['Exited'] = y_train
print(df_under.Exited.value_counts())
count_class0,count_class1 = df_under.Exited.value_counts()

df_under_0 = df_under[df_under.Exited==0]
df_under_1 = df_under[df_under.Exited==1]

df_under_0_new = df_under_0.sample(count_class1)
final_df = pd.concat([df_under_0_new,df_under_1],axis = 0)

x_train_under = final_df.drop('Exited',axis='columns')
y_train_under = final_df.Exited
y_train_under.value_counts()

0    6367
1    1633
Name: Exited, dtype: int64


0    1633
1    1633
Name: Exited, dtype: int64

In [28]:
y_pred = ANN(x_train_under,x_test,y_train_under,y_test,'binary_crossentropy',-1)

Epoch 1/100
103/103 [==============================] - 1s 3ms/step - loss: 0.6529 - accuracy: 0.6216
Epoch 2/100
103/103 [==============================] - 0s 3ms/step - loss: 0.6078 - accuracy: 0.6758
Epoch 3/100
103/103 [==============================] - 0s 3ms/step - loss: 0.5820 - accuracy: 0.6990
Epoch 4/100
103/103 [==============================] - 0s 3ms/step - loss: 0.5656 - accuracy: 0.7064
Epoch 5/100
103/103 [==============================] - 0s 3ms/step - loss: 0.5423 - accuracy: 0.7232
Epoch 6/100
103/103 [==============================] - 0s 3ms/step - loss: 0.5240 - accuracy: 0.7437
Epoch 7/100
103/103 [==============================] - 0s 3ms/step - loss: 0.5091 - accuracy: 0.7495
Epoch 8/100
103/103 [==============================] - 0s 3ms/step - loss: 0.4965 - accuracy: 0.7560
Epoch 9/100
103/103 [==============================] - 0s 3ms/step - loss: 0.4898 - accuracy: 0.7612
Epoch 10/100
103/103 [==============================] - 0s 3ms/step - loss: 0.4878 - accura

# METHOD 2: OVERSAMPLING_by Duplication

In [15]:
df_over = x_train.copy()
df_over['Exited'] = y_train
count_0,count_1 =  df_over['Exited'].value_counts()
df_over['Exited'].value_counts()

0    6367
1    1633
Name: Exited, dtype: int64

In [18]:
df_over_class0 = df_over[df_over.Exited==0] 
df_over_class1 = df_over[df_over.Exited==1] 
df_over_class1_update = df_over_class1.sample(count_0,replace=True)
df_over_final = pd.concat([df_over_class0,df_over_class1_update],axis=0)
df_over_final.Exited.value_counts()

0    6367
1    6367
Name: Exited, dtype: int64

In [19]:
x_train_over_dup = df_over_final.drop("Exited",axis='columns')
y_train_over_dup = df_over_final.Exited
y_pred_over_dup = ANN(x_train_over_dup,x_test,y_train_over_dup,y_test,'binary_crossentropy',-1)

Epoch 1/100
398/398 [==============================] - 1s 1ms/step - loss: 0.5999 - accuracy: 0.6766
Epoch 2/100
398/398 [==============================] - 1s 1ms/step - loss: 0.5305 - accuracy: 0.7335
Epoch 3/100
398/398 [==============================] - 1s 1ms/step - loss: 0.4918 - accuracy: 0.7573
Epoch 4/100
398/398 [==============================] - 1s 1ms/step - loss: 0.4774 - accuracy: 0.7652
Epoch 5/100
398/398 [==============================] - 1s 1ms/step - loss: 0.4689 - accuracy: 0.7698
Epoch 6/100
398/398 [==============================] - 1s 1ms/step - loss: 0.4633 - accuracy: 0.7686
Epoch 7/100
398/398 [==============================] - 1s 1ms/step - loss: 0.4579 - accuracy: 0.7727
Epoch 8/100
398/398 [==============================] - 1s 1ms/step - loss: 0.4541 - accuracy: 0.7752
Epoch 9/100
398/398 [==============================] - 1s 1ms/step - loss: 0.4498 - accuracy: 0.7802
Epoch 10/100
398/398 [==============================] - 1s 1ms/step - loss: 0.4459 - accura

398/398 [==============================] - 1s 1ms/step - loss: 0.2931 - accuracy: 0.8721
Epoch 82/100
398/398 [==============================] - 1s 1ms/step - loss: 0.2906 - accuracy: 0.8755
Epoch 83/100
398/398 [==============================] - 1s 1ms/step - loss: 0.2900 - accuracy: 0.8773
Epoch 84/100
398/398 [==============================] - 1s 1ms/step - loss: 0.2909 - accuracy: 0.8777
Epoch 85/100
398/398 [==============================] - 1s 1ms/step - loss: 0.2894 - accuracy: 0.8752
Epoch 86/100
398/398 [==============================] - 1s 1ms/step - loss: 0.2848 - accuracy: 0.8792
Epoch 87/100
398/398 [==============================] - 1s 1ms/step - loss: 0.2834 - accuracy: 0.8811
Epoch 88/100
398/398 [==============================] - 1s 1ms/step - loss: 0.2818 - accuracy: 0.8805
Epoch 89/100
398/398 [==============================] - 1s 1ms/step - loss: 0.2825 - accuracy: 0.8839
Epoch 90/100
398/398 [==============================] - 1s 1ms/step - loss: 0.2790 - accuracy: 

Classification report Initial:                
    
                 precision    recall  f1-score   support

           0       0.88      0.94      0.91      1596
           1       0.68      0.49      0.57       404

    accuracy                           0.85      2000
    macro avg       0.78      0.71      0.74      2000
    weighted avg    0.84      0.85      0.84      2000


# METHOD 3: OVERSAMPLING_by SMOTE

In [21]:
df_smote = x_train.copy()
df_smote['Exited'] = y_train
count_class0,count_class1 = df_smote.Exited.value_counts()
df_smote.Exited.value_counts()

0    6367
1    1633
Name: Exited, dtype: int64

In [22]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
x_sm,y_sm = smote.fit_resample(x_train,y_train)

y_sm.value_counts()

0    6367
1    6367
Name: Exited, dtype: int64

In [26]:
y_pred_smote = ANN(x_sm,x_test,y_sm,y_test,'binary_crossentropy',-1)

Epoch 1/100
398/398 [==============================] - 1s 1ms/step - loss: 0.6077 - accuracy: 0.6672
Epoch 2/100
398/398 [==============================] - 1s 1ms/step - loss: 0.5327 - accuracy: 0.7365
Epoch 3/100
398/398 [==============================] - 1s 1ms/step - loss: 0.4818 - accuracy: 0.7621
Epoch 4/100
398/398 [==============================] - 1s 1ms/step - loss: 0.4599 - accuracy: 0.7771
Epoch 5/100
398/398 [==============================] - 1s 1ms/step - loss: 0.4482 - accuracy: 0.7855
Epoch 6/100
398/398 [==============================] - 1s 1ms/step - loss: 0.4398 - accuracy: 0.7890
Epoch 7/100
398/398 [==============================] - 1s 1ms/step - loss: 0.4335 - accuracy: 0.7937
Epoch 8/100
398/398 [==============================] - 1s 1ms/step - loss: 0.4278 - accuracy: 0.7979
Epoch 9/100
398/398 [==============================] - 1s 1ms/step - loss: 0.4232 - accuracy: 0.8020
Epoch 10/100
398/398 [==============================] - 1s 1ms/step - loss: 0.4213 - accura

398/398 [==============================] - 1s 1ms/step - loss: 0.2716 - accuracy: 0.8868
Epoch 82/100
398/398 [==============================] - 1s 1ms/step - loss: 0.2718 - accuracy: 0.8842
Epoch 83/100
398/398 [==============================] - 1s 1ms/step - loss: 0.2711 - accuracy: 0.8857
Epoch 84/100
398/398 [==============================] - 1s 1ms/step - loss: 0.2701 - accuracy: 0.8862
Epoch 85/100
398/398 [==============================] - 1s 1ms/step - loss: 0.2702 - accuracy: 0.8861
Epoch 86/100
398/398 [==============================] - 1s 1ms/step - loss: 0.2685 - accuracy: 0.8857
Epoch 87/100
398/398 [==============================] - 1s 1ms/step - loss: 0.2663 - accuracy: 0.8866
Epoch 88/100
398/398 [==============================] - 1s 1ms/step - loss: 0.2668 - accuracy: 0.8901
Epoch 89/100
398/398 [==============================] - 1s 1ms/step - loss: 0.2679 - accuracy: 0.8853
Epoch 90/100
398/398 [==============================] - 1s 1ms/step - loss: 0.2671 - accuracy: 

Classification report Initial:                
    
                 precision    recall  f1-score   support

           0       0.88      0.94      0.91      1596
           1       0.68      0.49      0.57       404

    accuracy                           0.85      2000
    macro avg       0.78      0.71      0.74      2000
    weighted avg    0.84      0.85      0.84      2000


# METHOD 4: ENSEMBLE

In [30]:
ensemble_df = x_train.copy()
ensemble_df['Exited'] = y_train
count0,count1 = ensemble_df.Exited.value_counts()
ensemble_df.Exited.value_counts()

0    6367
1    1633
Name: Exited, dtype: int64

In [32]:
ensemble_df_class0 = ensemble_df[ensemble_df.Exited==0]
ensemble_df_class1 = ensemble_df[ensemble_df.Exited==1]

In [56]:
def get_train_batch(df_majority,df_minority,start,end):
    df_train = pd.concat([df_majority[start:end],df_minority],axis=0)
    x_train = df_train.drop("Exited",axis='columns')
    y_train = df_train.Exited
    return x_train,y_train

In [57]:
x_train_1,y_train_1 =get_train_batch(ensemble_df_class0,ensemble_df_class1,0,count1)
y_pred_1 = ANN(x_train_1,x_test,y_train_1,y_test,'binary_crossentropy',-1)

Epoch 1/100
103/103 [==============================] - 1s 1ms/step - loss: 0.6460 - accuracy: 0.6347
Epoch 2/100
103/103 [==============================] - 0s 1ms/step - loss: 0.6035 - accuracy: 0.6751
Epoch 3/100
103/103 [==============================] - 0s 1ms/step - loss: 0.5797 - accuracy: 0.6969
Epoch 4/100
103/103 [==============================] - 0s 1ms/step - loss: 0.5578 - accuracy: 0.7140
Epoch 5/100
103/103 [==============================] - 0s 1ms/step - loss: 0.5338 - accuracy: 0.7312
Epoch 6/100
103/103 [==============================] - 0s 1ms/step - loss: 0.5184 - accuracy: 0.7422
Epoch 7/100
103/103 [==============================] - 0s 1ms/step - loss: 0.5028 - accuracy: 0.7572
Epoch 8/100
103/103 [==============================] - 0s 1ms/step - loss: 0.4931 - accuracy: 0.7670
Epoch 9/100
103/103 [==============================] - 0s 1ms/step - loss: 0.4892 - accuracy: 0.7722
Epoch 10/100
103/103 [==============================] - 0s 1ms/step - loss: 0.4864 - accura

103/103 [==============================] - 0s 1ms/step - loss: 0.3985 - accuracy: 0.8184
Epoch 82/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3927 - accuracy: 0.8163
Epoch 83/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3886 - accuracy: 0.8227
Epoch 84/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3922 - accuracy: 0.8249
Epoch 85/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3875 - accuracy: 0.8264
Epoch 86/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3863 - accuracy: 0.8255
Epoch 87/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3807 - accuracy: 0.8279
Epoch 88/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3841 - accuracy: 0.8264
Epoch 89/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3801 - accuracy: 0.8224
Epoch 90/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3805 - accuracy: 

In [58]:
x_train_2,y_train_2 =get_train_batch(ensemble_df_class0,ensemble_df_class1,count1,count1*2)
y_pred_2 = ANN(x_train_2,x_test,y_train_2,y_test,'binary_crossentropy',-1)

Epoch 1/100
103/103 [==============================] - 1s 1ms/step - loss: 0.6448 - accuracy: 0.6402
Epoch 2/100
103/103 [==============================] - 0s 1ms/step - loss: 0.6095 - accuracy: 0.6739
Epoch 3/100
103/103 [==============================] - 0s 1ms/step - loss: 0.5899 - accuracy: 0.6846
Epoch 4/100
103/103 [==============================] - 0s 2ms/step - loss: 0.5754 - accuracy: 0.6969
Epoch 5/100
103/103 [==============================] - 0s 2ms/step - loss: 0.5659 - accuracy: 0.7064
Epoch 6/100
103/103 [==============================] - 0s 2ms/step - loss: 0.5559 - accuracy: 0.7070
Epoch 7/100
103/103 [==============================] - 0s 1ms/step - loss: 0.5389 - accuracy: 0.7263
Epoch 8/100
103/103 [==============================] - 0s 1ms/step - loss: 0.5217 - accuracy: 0.7480
Epoch 9/100
103/103 [==============================] - 0s 1ms/step - loss: 0.5096 - accuracy: 0.7569
Epoch 10/100
103/103 [==============================] - 0s 1ms/step - loss: 0.4927 - accura

103/103 [==============================] - 0s 1ms/step - loss: 0.3899 - accuracy: 0.8163
Epoch 82/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3875 - accuracy: 0.8209
Epoch 83/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3892 - accuracy: 0.8166
Epoch 84/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3846 - accuracy: 0.8233
Epoch 85/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3899 - accuracy: 0.8190
Epoch 86/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3887 - accuracy: 0.8239
Epoch 87/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3901 - accuracy: 0.8200
Epoch 88/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3891 - accuracy: 0.8187
Epoch 89/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3844 - accuracy: 0.8181
Epoch 90/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3855 - accuracy: 

In [59]:
x_train_3,y_train_3 =get_train_batch(ensemble_df_class0,ensemble_df_class1,count1*2,count1*3)
y_pred_3 = ANN(x_train_3,x_test,y_train_3,y_test,'binary_crossentropy',-1)

Epoch 1/100
103/103 [==============================] - 1s 1ms/step - loss: 0.6446 - accuracy: 0.6375
Epoch 2/100
103/103 [==============================] - 0s 1ms/step - loss: 0.6070 - accuracy: 0.6730
Epoch 3/100
103/103 [==============================] - 0s 1ms/step - loss: 0.5841 - accuracy: 0.6926
Epoch 4/100
103/103 [==============================] - 0s 1ms/step - loss: 0.5636 - accuracy: 0.7107
Epoch 5/100
103/103 [==============================] - 0s 1ms/step - loss: 0.5468 - accuracy: 0.7250
Epoch 6/100
103/103 [==============================] - 0s 1ms/step - loss: 0.5264 - accuracy: 0.7382
Epoch 7/100
103/103 [==============================] - 0s 1ms/step - loss: 0.5085 - accuracy: 0.7465
Epoch 8/100
103/103 [==============================] - 0s 1ms/step - loss: 0.4948 - accuracy: 0.7609
Epoch 9/100
103/103 [==============================] - 0s 1ms/step - loss: 0.4894 - accuracy: 0.7615
Epoch 10/100
103/103 [==============================] - 0s 1ms/step - loss: 0.4780 - accura

103/103 [==============================] - 0s 1ms/step - loss: 0.3827 - accuracy: 0.8218
Epoch 82/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3839 - accuracy: 0.8218
Epoch 83/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3795 - accuracy: 0.8255
Epoch 84/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3809 - accuracy: 0.8252
Epoch 85/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3794 - accuracy: 0.8227
Epoch 86/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3826 - accuracy: 0.8227
Epoch 87/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3913 - accuracy: 0.8172
Epoch 88/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3796 - accuracy: 0.8282
Epoch 89/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3769 - accuracy: 0.8246
Epoch 90/100
103/103 [==============================] - 0s 1ms/step - loss: 0.3720 - accuracy: 

In [60]:
x_train_4,y_train_4 =get_train_batch(ensemble_df_class0,ensemble_df_class1,count1*3,count0)
y_pred_4 = ANN(x_train_4,x_test,y_train_4,y_test,'binary_crossentropy',-1)

Epoch 1/100
97/97 [==============================] - 1s 1ms/step - loss: 0.6494 - accuracy: 0.6230
Epoch 2/100
97/97 [==============================] - 0s 1ms/step - loss: 0.6140 - accuracy: 0.6701
Epoch 3/100
97/97 [==============================] - 0s 1ms/step - loss: 0.5871 - accuracy: 0.6904
Epoch 4/100
97/97 [==============================] - 0s 1ms/step - loss: 0.5697 - accuracy: 0.6962
Epoch 5/100
97/97 [==============================] - 0s 2ms/step - loss: 0.5520 - accuracy: 0.7198
Epoch 6/100
97/97 [==============================] - 0s 2ms/step - loss: 0.5327 - accuracy: 0.7343
Epoch 7/100
97/97 [==============================] - 0s 1ms/step - loss: 0.5140 - accuracy: 0.7401
Epoch 8/100
97/97 [==============================] - 0s 1ms/step - loss: 0.4982 - accuracy: 0.7549
Epoch 9/100
97/97 [==============================] - 0s 2ms/step - loss: 0.4915 - accuracy: 0.7556
Epoch 10/100
97/97 [==============================] - 0s 2ms/step - loss: 0.4839 - accuracy: 0.7665
Epoch 11/

97/97 [==============================] - 0s 1ms/step - loss: 0.4009 - accuracy: 0.8039
Epoch 84/100
97/97 [==============================] - 0s 1ms/step - loss: 0.4002 - accuracy: 0.8107
Epoch 85/100
97/97 [==============================] - 0s 1ms/step - loss: 0.3963 - accuracy: 0.8104
Epoch 86/100
97/97 [==============================] - 0s 1ms/step - loss: 0.3943 - accuracy: 0.8130
Epoch 87/100
97/97 [==============================] - 0s 1ms/step - loss: 0.3968 - accuracy: 0.8107
Epoch 88/100
97/97 [==============================] - 0s 1ms/step - loss: 0.3954 - accuracy: 0.8133
Epoch 89/100
97/97 [==============================] - 0s 1ms/step - loss: 0.3949 - accuracy: 0.8114
Epoch 90/100
97/97 [==============================] - 0s 1ms/step - loss: 0.3916 - accuracy: 0.8094
Epoch 91/100
97/97 [==============================] - 0s 1ms/step - loss: 0.3924 - accuracy: 0.8114
Epoch 92/100
97/97 [==============================] - 0s 1ms/step - loss: 0.3898 - accuracy: 0.8133
Epoch 93/100


In [61]:
y_pred_final = y_pred_1.copy()
for i in range(len(y_pred_1)):
    sum = y_pred_1[i]+y_pred_2[i]+y_pred_3[i]+y_pred_4[i]
    if sum >  1:
        y_pred_final[i]=1
    else:
        y_pred_final[i]=0

In [62]:
print(classification_report(y_test,y_pred_final))

              precision    recall  f1-score   support

           0       0.93      0.76      0.84      1596
           1       0.45      0.77      0.57       404

    accuracy                           0.76      2000
   macro avg       0.69      0.77      0.70      2000
weighted avg       0.83      0.76      0.78      2000



Classification report Initial:                
    
                 precision    recall  f1-score   support

           0       0.88      0.94      0.91      1596
           1       0.68      0.49      0.57       404

    accuracy                           0.85      2000
    macro avg       0.78      0.71      0.74      2000
    weighted avg    0.84      0.85      0.84      2000
